In [1]:
import pandas as pd
import numpy as np

### Data

In [2]:
course = pd.read_csv("course.csv") 
course_element = pd.read_csv("course_element.csv") 
course_module = pd.read_csv("course_module.csv") 
sol_log = pd.read_csv("solution_log.csv") 
u_course_progress = pd.read_csv("user_course_progress.csv") 
u_element_progress = pd.read_csv("user_element_progress.csv") 
u_module_progress = pd.read_csv("user_module_progress.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
u_course_ids = pd.read_csv("user_course_ids.csv")

### Preprocessing (убираем неактивных юзеров в курсах)

In [3]:
# to_csv
u_tasks_done = u_module_progress.groupby(['user_id', 'course_id'])['tasks_done'].sum().reset_index()
user_course_ids = u_tasks_done[u_tasks_done['tasks_done'] > 0][['user_id', 'course_id']]

user_course_ids.reset_index(drop=True).to_csv('user_course_ids.csv', index=False)

In [4]:
# filter
u_course_progress = u_course_progress.merge(u_course_ids, on=['user_id', 'course_id'])
u_module_progress = u_module_progress.merge(u_course_ids, on=['user_id', 'course_id'])
u_element_progress = u_element_progress.merge(u_course_ids, on=['user_id', 'course_id'])

---

### user_max_delta_sub_time

In [5]:
u_course_regularity = u_element_progress.merge(sol_log, how='left', left_on='id', 
                     right_on='element_progress_id')[['user_id', 'submission_time', 'course_id', 'verdict']]

In [6]:
u_course_regularity['submission_time'] = pd.to_datetime(u_course_regularity['submission_time'])

In [7]:
u_course_regularity.sort_values('submission_time', inplace=True)

In [8]:
u_course_regularity['delta'] = u_course_regularity.groupby(['user_id', 'course_id'])['submission_time'].diff()

In [9]:
u_course_regularity['delta'] = u_course_regularity['delta'].apply(lambda x: round(x.total_seconds(), 2))

In [12]:
u_max_delta_sub_time = u_course_regularity.groupby(['user_id', 'course_id'])['delta'].max().reset_index()

In [13]:
#to_csv
u_max_delta_sub_time.to_csv('user_max_delta_sub_time.csv', index=False)

---

### периоды активности

In [10]:
u_course_regularity['is_new_period'] = u_course_regularity['delta'] > (3600 * 24)

In [11]:
u_course_regularity['period_of_activity'] = u_course_regularity.groupby(
    ['user_id', 'course_id'])['is_new_period'].apply(lambda x: x.cumsum())

In [12]:
# verdict == 'ok'
grouped = u_course_regularity[u_course_regularity['verdict'] == 'ok'].groupby(
    ['user_id', 'course_id', 'period_of_activity'])

In [13]:
u_course_period_activity = grouped['verdict'].apply(lambda x: x.count()).reset_index()

In [14]:
max_done = u_course_period_activity.groupby(['user_id', 'course_id'])['verdict'].max().reset_index()
mean_done = u_course_period_activity.groupby(['user_id', 'course_id'])['verdict'].mean().reset_index()
med_done = u_course_period_activity.groupby(['user_id', 'course_id'])['verdict'].median().reset_index()

In [15]:
u_course_activity = u_course_ids.merge(max_done, on=['user_id', 'course_id'])
u_course_activity = u_course_activity.merge(mean_done, on=['user_id', 'course_id'])
u_course_activity = u_course_activity.merge(med_done, on=['user_id', 'course_id'])

In [18]:
# all tries
grouped = u_course_regularity.groupby(['user_id', 'course_id', 'period_of_activity'])
u_course_period_activity = grouped['verdict'].apply(lambda x: x.count()).reset_index()

max_done = u_course_period_activity.groupby(['user_id', 'course_id'])['verdict'].max().reset_index()
mean_done = u_course_period_activity.groupby(['user_id', 'course_id'])['verdict'].mean().reset_index()
med_done = u_course_period_activity.groupby(['user_id', 'course_id'])['verdict'].median().reset_index()

u_course_activity = u_course_activity.merge(max_done, on=['user_id', 'course_id'])
u_course_activity = u_course_activity.merge(mean_done, on=['user_id', 'course_id'])
u_course_activity = u_course_activity.merge(med_done, on=['user_id', 'course_id'])

u_course_activity.set_axis(['user_id', 'course_id', 'max_done', 'mean_done', 'med_done', 
                            'max_tries', 'mean_tries', 'med_tries'], axis='columns', inplace=True)

In [19]:
u_course_activity

,user_id,course_id,max_done,mean_done,med_done,max_tries,mean_tries,med_tries
0,1034,3,16,7.842105,8.0,22,9.789474,8.0
1,3971,3,129,37.250000,9.0,425,112.750000,11.0
2,100119700110000264,3,2,2.000000,2.0,2,2.000000,2.0
3,100119700110000274,3,39,15.500000,10.5,57,21.750000,13.5
4,100119700110000277,35,81,28.333333,19.5,102,33.666667,20.5
...,...,...,...,...,...,...,...,...
16598,100120190110554361,3,1,1.000000,1.0,3,3.000000,3.0
16599,100120190110586064,3,4,4.000000,4.0,4,4.000000,4.0
16600,100120190110590055,89,54,26.000000,22.0,70,30.833333,25.0
16601,100120190110592083,3,77,16.555556,9.0,119,27.222222,11.0


In [21]:
u_course_regularity[u_course_regularity['is_new_period'] == True]

,user_id,submission_time,course_id,verdict,delta,is_new_period,period_of_activity
3828258,100119970110044783,2019-03-15 14:59:41.526184,3,ok,92397.52,True,1
7399640,100119970110044783,2019-03-15 15:00:28.373502,89,ok,92460.23,True,1
1617252,100120060110286686,2019-03-17 10:02:21.192412,3,ok,146217.73,True,1
3617161,100120020110158421,2019-03-17 11:51:17.829953,3,wrong,88291.37,True,1
1237226,100120020110158421,2019-03-17 11:58:13.452105,35,ok,88707.00,True,1
...,...,...,...,...,...,...,...
2103373,100119760111027282,2020-11-30 18:41:14.227387,237,wrong,87306.04,True,13
4016192,100120040110043964,2020-11-30 18:52:14.854493,237,ok,15139105.16,True,17
4038627,100120060111114061,2020-11-30 18:56:11.171693,237,ok,90606.67,True,20
3136465,100120020111600826,2020-11-30 19:11:30.278662,237,wrong,102833.87,True,4


In [28]:
u_course_period_info_1 = u_course_regularity.groupby(
    ['user_id', 'course_id', 'period_of_activity'])['submission_time'].min().reset_index(name='min_sub_time')
u_course_period_info_2 = u_course_regularity.groupby(
    ['user_id', 'course_id', 'period_of_activity'])['submission_time'].max().reset_index(name='max_sub_time')

u_course_period_info = u_course_period_info_1.merge(
    u_course_period_info_2, on=['user_id', 'course_id', 'period_of_activity'])

In [55]:
u_course_period_info[u_course_period_info['user_id'] == 100119700110000264]

,user_id,course_id,period_of_activity,min_sub_time,max_sub_time
23,100119700110000264,3,0,2019-03-26 08:03:29.723591,2019-03-26 10:58:41.133163


In [49]:
u_course_delta_1 = u_course_period_info.groupby(['user_id', 'course_id']).apply(
    lambda x: (x['min_sub_time'][1:].array - x['max_sub_time'][:-1].array).mean()).reset_index(name='mean_period_deltas')
u_course_delta_2 = u_course_period_info.groupby(['user_id', 'course_id']).apply(
    lambda x: (x['min_sub_time'][1:].array - x['max_sub_time'][:-1].array).median()).reset_index(name='median_period_deltas')

In [50]:
u_course_activity = u_course_activity.merge(u_course_delta_1, on=['user_id', 'course_id'])
u_course_activity = u_course_activity.merge(u_course_delta_2, on=['user_id', 'course_id'])

In [52]:
u_course_activity['mean_period_deltas'] = u_course_activity['mean_period_deltas'].apply(lambda x: round(x.total_seconds(), 2))
u_course_activity['median_period_deltas'] = u_course_activity['median_period_deltas'].apply(lambda x: round(x.total_seconds(), 2))

In [57]:
u_course_activity

,user_id,course_id,max_done,mean_done,med_done,max_tries,mean_tries,med_tries,mean_period_deltas,median_period_deltas
0,1034,3,16,7.842105,8.0,22,9.789474,8.0,326703.28,202476.36
1,3971,3,129,37.250000,9.0,425,112.750000,11.0,1615713.55,255344.21
2,100119700110000264,3,2,2.000000,2.0,2,2.000000,2.0,NaN,NaN
3,100119700110000274,3,39,15.500000,10.5,57,21.750000,13.5,2229105.49,1360963.60
4,100119700110000277,35,81,28.333333,19.5,102,33.666667,20.5,295227.71,167720.01
...,...,...,...,...,...,...,...,...,...,...
16598,100120190110554361,3,1,1.000000,1.0,3,3.000000,3.0,NaN,NaN
16599,100120190110586064,3,4,4.000000,4.0,4,4.000000,4.0,NaN,NaN
16600,100120190110590055,89,54,26.000000,22.0,70,30.833333,25.0,1864087.22,523601.35
16601,100120190110592083,3,77,16.555556,9.0,119,27.222222,11.0,435934.34,204341.03


In [56]:
u_course_activity[u_course_activity['mean_period_deltas'].isnull()].count()

user_id                 5762
course_id               5762
max_done                5762
mean_done               5762
med_done                5762
max_tries               5762
mean_tries              5762
med_tries               5762
mean_period_deltas         0
median_period_deltas       0
dtype: int64

---

### отношение решённых модулей(1 - без учёта advance, 2 - только advanced) / ( время с начало курса прохождения до последнего апдейта пользователем)

In [38]:
u_course_progress['time_updated'] = pd.to_datetime(u_course_progress['time_updated'])
u_course_progress['time_created'] = pd.to_datetime(u_course_progress['time_created'])

In [39]:
grouped = u_course_progress.groupby(['user_id', 'course_id'])

In [40]:
u_course_progress['time'] = u_course_progress['time_updated'] - u_course_progress['time_created']

In [45]:
u_course_progress['time'] = u_course_progress['time'].apply(lambda x: x.total_seconds())

In [46]:
sum_modules = grouped['modules_achieved'].sum().reset_index()
sum_adv_modules = grouped['advanced_modules_done'].sum().reset_index()

In [47]:
u_speed = sum_modules.merge(u_course_progress, 
                            on=['user_id', 'course_id'])[['user_id', 'course_id', 'modules_achieved_x', 'time']]
u_speed = u_speed.merge(sum_adv_modules, on=['user_id', 'course_id'])

In [48]:
u_speed['modules_achieved_1'] = u_speed['modules_achieved_x'] - u_speed['advanced_modules_done']

In [53]:
u_speed['speed'] = 1000000 * (u_speed['modules_achieved_x'] / u_speed['time'])
u_speed['speed_1'] = 1000000 * (u_speed['modules_achieved_1'] / u_speed['time'])
u_speed['speed_2'] = 1000000 * (u_speed['advanced_modules_done'] / u_speed['time'])

In [54]:
speeds_me = u_speed[['user_id', 'course_id', 'speed', 'speed_1', 'speed_2']]

---

### transferred (???)

In [36]:
u_element_progress[u_element_progress['achieve_reason'] == 'transferred']

,id,user_id,course_id,course_module_id,course_element_type,course_element_id,progress_current,is_achieved,hints_used,tries_count,module_progress_id,time_created,time_updated,time_closed,time_achieved,time_started,achieve_reason
1391,56066696,100120050111438874,89,1207,task,619,0.0,True,0,1.0,2956395,2020-07-29 13:05:51.540607,NaN,2020-07-29 13:11:50.318357,2020-07-29 13:11:50.318357,NaN,transferred
1392,56066695,100120050111438874,89,1207,task,618,0.0,True,0,1.0,2956395,2020-07-29 13:05:51.540607,NaN,2020-07-29 13:11:50.318357,2020-07-29 13:11:50.318357,NaN,transferred
1393,56066694,100120050111438874,89,1207,task,617,0.0,True,0,1.0,2956395,2020-07-29 13:05:51.540607,NaN,2020-07-29 13:11:50.318357,2020-07-29 13:11:50.318357,NaN,transferred
1394,56066693,100120050111438874,89,1207,task,616,0.0,True,0,1.0,2956395,2020-07-29 13:05:51.540607,NaN,2020-07-29 13:11:50.318357,2020-07-29 13:11:50.318357,NaN,transferred
1395,56066692,100120050111438874,89,1207,task,615,0.0,True,0,1.0,2956395,2020-07-29 13:05:51.540607,NaN,2020-07-29 13:11:50.318357,2020-07-29 13:11:50.318357,NaN,transferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274418,47178967,100120060110742614,89,1197,video,424,0.0,True,0,1.0,2471373,2020-05-13 13:03:11.932436,NaN,2020-05-13 13:03:32.274857,2020-05-13 13:03:32.274857,NaN,transferred
6274419,47178968,100120060110742614,89,1197,task,1966,1.0,True,0,1.0,2471373,2020-05-13 13:03:11.932436,NaN,2020-05-13 13:03:32.274857,2020-05-13 13:03:32.274857,NaN,transferred
6274428,47178961,100120060110742614,89,1197,video,351,0.0,True,0,1.0,2471373,2020-05-13 13:03:11.932436,NaN,2020-05-13 13:03:32.274857,2020-05-13 13:03:32.274857,NaN,transferred
6274429,47178962,100120060110742614,89,1197,task,622,1.0,True,0,1.0,2471373,2020-05-13 13:03:11.932436,NaN,2020-05-13 13:03:32.274857,2020-05-13 13:03:32.274857,NaN,transferred
